<div class="elfjS" data-track-load="description_content"><p>Given an <code>m x n</code> <code>board</code>&nbsp;of characters and a list of strings <code>words</code>, return <em>all words on the board</em>.</p>

<p>Each word must be constructed from letters of sequentially adjacent cells, where <strong>adjacent cells</strong> are horizontally or vertically neighboring. The same letter cell may not be used more than once in a word.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>
<img alt="" src="https://assets.leetcode.com/uploads/2020/11/07/search1.jpg" style="width: 322px; height: 322px;">
<pre><strong>Input:</strong> board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]], words = ["oath","pea","eat","rain"]
<strong>Output:</strong> ["eat","oath"]
</pre>

<p><strong class="example">Example 2:</strong></p>
<img alt="" src="https://assets.leetcode.com/uploads/2020/11/07/search2.jpg" style="width: 162px; height: 162px;">
<pre><strong>Input:</strong> board = [["a","b"],["c","d"]], words = ["abcb"]
<strong>Output:</strong> []
</pre>

<p>&nbsp;</p>
<p><strong>Constraints:</strong></p>

<ul>
	<li><code>m == board.length</code></li>
	<li><code>n == board[i].length</code></li>
	<li><code>1 &lt;= m, n &lt;= 12</code></li>
	<li><code>board[i][j]</code> is a lowercase English letter.</li>
	<li><code>1 &lt;= words.length &lt;= 3 * 10<sup>4</sup></code></li>
	<li><code>1 &lt;= words[i].length &lt;= 10</code></li>
	<li><code>words[i]</code> consists of lowercase English letters.</li>
	<li>All the strings of <code>words</code> are unique.</li>
</ul>
</div>

In [14]:
from typing import *
from functools import lru_cache

In [15]:
class Node:
    def __init__(self, character: str, is_terminal: bool):
        self.character = character
        self.is_terminal = is_terminal
        # a node can only have 26 children since there are only 26 lowercase letters
        self.children = [None] * 26

class Trie:
    def __init__(self):
        self.root = Node("", False)

    def insert_word(self, word: str):
        curr_node = self.root
        for i, c in enumerate(word):
            is_terminal = i == len(word) - 1
            child_idx = ord(c) - ord('a')
            if curr_node.children[child_idx] is None:
                next_node = Node(c, is_terminal)
                curr_node.children[child_idx] = next_node
            else:
                next_node = curr_node.children[child_idx]
                next_node.is_terminal |= is_terminal
            curr_node = next_node

    def find_prefix(self, prefix: str, start_node: Node) -> Optional[Node]:
        if start_node is not None:
            curr_node = start_node
        else:
            curr_node = self.root

        for i, c in enumerate(prefix):
            child_idx = ord(c) - ord('a')
            if curr_node.children[child_idx] is None:
                # this prefix does not exist in the trie
                return None
            curr_node = curr_node.children[child_idx]

        # the prefix existed in the Trie and we can return its node
        return curr_node

class Solution:
    def findWords(self, board: List[List[str]], words: List[str], verbose: bool = False) -> List[str]:
        m = len(board)
        n = len(board[0])
        # result is the set of words we have found on the board
        trie = Trie()
        word_idx: Dict[str, int] = {}
        word_found = [False] * len(words)
        for i, word in enumerate(words):
            # insert the word into the trie
            trie.insert_word(word)
            word_idx[word] = i

        # returns the 2D adjacent neighbors of some idx (i, j)
        get_neighbors = lambda i, j : ((i,j-1), (i,j+1), (i-1,j), (i+1,j))
        # returns True if (i, j) is out of range on the board
        out_of_range = lambda i, j : i < 0 or i >= m or j < 0 or j >= n

        def _slide(idx: Tuple[int, int], prefix: str, path: set, trie_node: Node):
            i, j = idx

            if trie_node.is_terminal:
                # this is a complete word we have found
                word_found[word_idx[prefix]] = True

            for a, b in get_neighbors(i, j):
                if out_of_range(a, b) or (a,b) in path:
                    # continue if we are out of range or
                    # this index was already visited in the path
                    continue
                next_node = trie.find_prefix(board[a][b], trie_node)
                if next_node is not None:
                    # next position (a, b) still forms a valid prefix, explore it and add it
                    # to the path
                    new_path = path.copy()
                    new_path.add((a,b))
                    _slide((a,b), prefix + board[a][b], new_path, next_node)

        for i in range(m):
            for j in range(n):
                # iterate through the board to start our prefix from different positions
                start_idx = (i, j)
                start_node = trie.find_prefix(board[i][j], None)
                if start_node is not None:
                    _slide(start_idx, board[i][j], set(((i,j),)), start_node)

        return [word for i, word in enumerate(words) if word_found[i]]

def main():
    test_cases = {
        "1": {
            "board": [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]],
            "words": ["oath","pea","eat","rain"],
            "expected": ["oath","eat"],
        },
        "2": {
            "board": [["a","b"],["c","d"]],
            "words": ["abcb"],
            "expected": [],
        },
        "3": {
            "board": [["a","a"]],
            "words": ["aaa"],
            "expected": [],
        },
        "4": {
            "board": [["a","a"],["a","a"]],
            "words": ["aaaaa"],
            "expected": [],
        },
    }

    solution = Solution()

    for tk, targs in test_cases.items():
        expected = targs.pop("expected", None)
        ret = solution.findWords(**targs, verbose=True)
        if expected is not None:
            passed = ret == expected
        else:
            passed = None
        print(f"test case {tk}: {targs}\nReturned: {ret}, Expected: {expected}\nPassed:{passed}\n")


main()

test case 1: {'board': [['o', 'a', 'a', 'n'], ['e', 't', 'a', 'e'], ['i', 'h', 'k', 'r'], ['i', 'f', 'l', 'v']], 'words': ['oath', 'pea', 'eat', 'rain']}
Returned: ['oath', 'eat'], Expected: ['oath', 'eat']
Passed:True

test case 2: {'board': [['a', 'b'], ['c', 'd']], 'words': ['abcb']}
Returned: [], Expected: []
Passed:True

test case 3: {'board': [['a', 'a']], 'words': ['aaa']}
Returned: [], Expected: []
Passed:True

test case 4: {'board': [['a', 'a'], ['a', 'a']], 'words': ['aaaaa']}
Returned: [], Expected: []
Passed:True



The above implementation exceeds the time limit on LC. We can make a simpler Trie and do backtracking more efficiently as follows:

In [16]:
class Solution:
    def findWords(self, board: List[List[str]], words: List[str], verbose: bool = False) -> List[str]:
        WORD_KEY = "$"

        trie = {}
        for word in words:
            node = trie
            for letter in word:
                # retrieve the next node; If not found, create a empty node.
                node = node.setdefault(letter, {}) # index node[letter], if does not exist, let node[letter]<-{}
            # mark the existence of a word in trie node
            node[WORD_KEY] = word

        rowNum = len(board)
        colNum = len(board[0])

        matchedWords = []

        def backtracking(row, col, parent):

            letter = board[row][col]
            # parent node in the Trie, we now get its child
            # by default the child is an empty dictionary if it does not exist
            currNode = parent[letter]

            # check if we find a match of word, remove terminal word key to indicate we have found
            # the word so that we do not add it again if found again
            word_match = currNode.pop(WORD_KEY, False)
            if word_match:
                matchedWords.append(word_match)

            # Before the EXPLORATION, mark the cell as visited
            board[row][col] = "#"

            # Explore the neighbors in 4 directions, i.e. up, right, down, left
            for rowOffset, colOffset in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
                newRow, newCol = row + rowOffset, col + colOffset
                if (
                    newRow < 0
                    or newRow >= rowNum
                    or newCol < 0
                    or newCol >= colNum
                ):
                    # next position is out of range
                    continue
                if not board[newRow][newCol] in currNode:
                    # Don't explore if the next position is not in the Trie.
                    # Simple checks like this are more efficient than using base cases at the
                    # start of a recursive function.
                    continue
                backtracking(newRow, newCol, currNode)

            # End of EXPLORATION, we restore the cell
            # NOTE: This shows that we can mutate the board and simply reset it in backtracking
            board[row][col] = letter

            # Optimization: incrementally remove the matched leaf node in Trie.
            # NOTE: Since our Trie has explored this path, we can safely remove the letter
            # as we do not need to explore it again.
            if not currNode:
                parent.pop(letter)

        for row in range(rowNum):
            for col in range(colNum):
                # starting from each of the cells
                if board[row][col] in trie:
                    backtracking(row, col, trie)

        return matchedWords

def main():
    test_cases = {
        "1": {
            "board": [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]],
            "words": ["oath","pea","eat","rain"],
            "expected": ["oath","eat"],
        },
        "2": {
            "board": [["a","b"],["c","d"]],
            "words": ["abcb"],
            "expected": [],
        },
        "3": {
            "board": [["a","a"]],
            "words": ["aaa"],
            "expected": [],
        },
        "4": {
            "board": [["a","a"],["a","a"]],
            "words": ["aaaaa"],
            "expected": [],
        },
    }

    solution = Solution()

    for tk, targs in test_cases.items():
        expected = targs.pop("expected", None)
        ret = solution.findWords(**targs, verbose=True)
        if expected is not None:
            passed = ret == expected
        else:
            passed = None
        print(f"test case {tk}: {targs}\nReturned: {ret}, Expected: {expected}\nPassed:{passed}\n")


main()

test case 1: {'board': [['o', 'a', 'a', 'n'], ['e', 't', 'a', 'e'], ['i', 'h', 'k', 'r'], ['i', 'f', 'l', 'v']], 'words': ['oath', 'pea', 'eat', 'rain']}
Returned: ['oath', 'eat'], Expected: ['oath', 'eat']
Passed:True

test case 2: {'board': [['a', 'b'], ['c', 'd']], 'words': ['abcb']}
Returned: [], Expected: []
Passed:True

test case 3: {'board': [['a', 'a']], 'words': ['aaa']}
Returned: [], Expected: []
Passed:True

test case 4: {'board': [['a', 'a'], ['a', 'a']], 'words': ['aaaaa']}
Returned: [], Expected: []
Passed:True

